In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import pandas as pd
from functional import seq
from tqdm.auto import tqdm

from utils import process_viztrace_json

In [ ]:
files = sorted(Path('scripts/benchmark_jsons').glob('*.json'))
files[0]

In [ ]:
df_mems = []
for file in tqdm(files):
    try:
        df_mem, df_info = process_viztrace_json(file)
        df_mem[["toolkit", "index", "nb_traces", "n", "n_out", "headless"]] = (
            seq(file.name.split(".")[0].split("__"))
                .map(lambda x: x.split("=")[-1])
                .to_list()
        )
        df_mem[['index', 'nb_traces', 'n', 'n_out']] = df_mem[
            ['index', 'nb_traces', 'n', 'n_out']].astype(int)
        df_mem.loc[:df_info[df_info.name.str.contains('constructed image')].index[0],
        'phase'] = 'image construction'
        df_mem.loc[df_info[df_info.name.str.contains('constructed image')].index[0]:,
        'phase'] = 'rendering'
        df_mems.append(df_mem.reset_index())
    except Exception as e:
        print(file)
        print(e)

In [ ]:
df_mem_tot = pd.concat(df_mems, ignore_index=True)
df_mem_tot['args.rss'] /= 1e6
df_mem_tot.rename(columns={'args.rss': 'memory (MB)'}, inplace=True)

In [ ]:
df_mem_tot.head(3)

In [ ]:
df_mem_viz = (
    df_mem_tot.groupby(["toolkit", "index", "nb_traces"])[["ts", "memory (MB)"]]
        .max()
        .reset_index()
        # .groupby(["toolkit", "nb_traces"])[["ts", "memory (MB)"]]
        # .apply(
        #     lambda x: pd.Series(
        #         data=x.mean().values.tolist() + (20 * x.std()).values.tolist(),
        #         index=("mean_" + x.columns).tolist() + ("std_" + x.columns).tolist(),
        #     )
        # )
        .reset_index()
)

In [ ]:
import seaborn as sns

sns.set_context('talk')

In [ ]:
sns.relplot(
    data=df_mem_viz,
    kind='line',
    hue='toolkit',
    x='nb_traces',
    y='memory (MB)',
    height=9,
    palette='Set2'
)

In [ ]:
df_mem_viz.head(3)

In [ ]:
df_mem_viz['ts'] = df_mem_viz['ts'].dt.total_seconds()

In [ ]:
g = sns.relplot(
    data=df_mem_viz,
    kind='line',
    hue='toolkit',
    x='nb_traces',
    y='ts',
    height=8,
    palette='Set2'
)

# g.ax.set_yscale('log')

In [ ]:
# df_mem_viz = (
#     df_mem_tot.groupby(["toolkit", "index", "nb_traces"])[["ts", "memory (MB)"]]
#     .max()
#     .reset_index()
#     .groupby(["toolkit", "nb_traces"])[["ts", "memory (MB)"]]
#     .apply(
#         lambda x: pd.Series(
#             data=x.mean().values.tolist() + (20 * x.std()).values.tolist(),
#             index=("mean_" + x.columns).tolist() + ("std_" + x.columns).tolist(),
#         )
#     )
#     .reset_index()
# )